In [ ]:
import sys
import numpy as np
import pandas as pd
import pickle
from flask import Flask, render_template,request
from keras.models import load_model, model_from_json
import keras
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils
from tensorflow.python.framework import ops

graph = ops.get_default_graph()

def unpack(model, training_config, weights):
    restored_model = deserialize(model)
    if training_config is not None:
        restored_model.compile(
            **saving_utils.compile_args_from_training_config(
                training_config
            )
        )
    restored_model.set_weights(weights)
    return restored_model


app = Flask(__name__)

logreg = pickle.load(open('models/logreg.pkl', 'rb'))
dt_entropy = pickle.load(open('models/dt_gini.pkl', 'rb'))
SVC_linear_kernel = pickle.load(open('models/SVC_linear_kernel.pkl', 'rb'))
random_forest= pickle.load(open('models/random_forest.pkl', 'rb'))

@app.route('/predict', methods=['POST'])
def home():

    var1=request.form["var1"]

    var2=request.form["var2"]

    var3=request.form["var3"]

    model=request.form["model"]

    X= np.array([[var1,var2,var3]])
    X = pd.DataFrame(X, columns = ['Column_A','Column_B','Column_C'])

    if(model == "Logistic Regression") :
        clf = logreg 
    elif(model == "Decision Tree") :
        clf = dt_entropy
    elif(model == "SVM") :
        clf = SVC_linear_kernel
    elif(model == "Random Forest") :
        clf = random_forest
    elif(model == "MLP") :
        
            global graph
            with graph.as_default():
                MLP= pickle.load(open('models/MLP.pkl', 'rb'))
                pred=(MLP.predict(X));
        
            pred = (pred > 0.5)
            if(pred):
                pred=1
            else:
                pred=0       
    
    
    if(model != "MLP") :
        pred = clf.predict(X)      
        
    if(pred==1):
        output="malignant"
    else:
        output="benign"

    return render_template('base.html', prediction_text='The tumor is {}.'.format(output))
    
@app.route('/', methods=['GET'])
def reset():
    return render_template('base.html')

if __name__ == '__main__':
   app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


C:\Users\User\anaconda3\envs\statistics\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\User\anaconda3\envs\statistics\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\User\anaconda3\envs\statistics\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\User\anaconda3\envs\statistics\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.23.2 when using